In [1]:

import os
import sys

import numpy as np
import pandas as pd
from scipy import stats


import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as colors

from datetime import datetime
import time
import warnings
import psycopg2
# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

np.set_printoptions(suppress=True)

D:\anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
kfold_rmse = pd.read_csv("../../Data/kflod-rmse.csv")

In [3]:
kfold_rmse

,testingset,FCN2,FCN1,CNN2,CNN6,CNN7,CNN5,CNN3,CNN1,LSTM2,LSTM4,CNN4,LSTM1,LSTM3
0,1_,15.93,15.96,16.03,15.87,15.79,16.09,16.12,15.83,16.67,16.27,16.28,16.43,16.80
1,2_,16.39,16.08,16.25,16.04,16.22,16.03,16.49,16.12,17.06,16.72,16.00,16.53,16.81
2,3_,16.88,16.83,16.54,16.71,16.60,16.64,16.88,16.95,17.44,16.86,16.64,16.66,17.41
3,4_,16.74,16.46,16.02,16.54,16.83,16.59,16.17,16.69,17.06,16.53,16.09,16.60,17.28
4,5_,16.87,16.68,16.16,16.35,16.16,16.14,16.58,16.07,17.06,16.69,15.90,16.42,17.46


In [4]:
kfold_rmse.index = ['testingset1','testingset2','testingset3','testingset4','testingset5']

In [5]:
kfold_rmse = kfold_rmse.drop('testingset',axis=1)

In [6]:
kfold_rmse

,FCN2,FCN1,CNN2,CNN6,CNN7,CNN5,CNN3,CNN1,LSTM2,LSTM4,CNN4,LSTM1,LSTM3
testingset1,15.93,15.96,16.03,15.87,15.79,16.09,16.12,15.83,16.67,16.27,16.28,16.43,16.80
testingset2,16.39,16.08,16.25,16.04,16.22,16.03,16.49,16.12,17.06,16.72,16.00,16.53,16.81
testingset3,16.88,16.83,16.54,16.71,16.60,16.64,16.88,16.95,17.44,16.86,16.64,16.66,17.41
testingset4,16.74,16.46,16.02,16.54,16.83,16.59,16.17,16.69,17.06,16.53,16.09,16.60,17.28
testingset5,16.87,16.68,16.16,16.35,16.16,16.14,16.58,16.07,17.06,16.69,15.90,16.42,17.46


In [47]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2

#Suppose that the test accuracy of each network for five different test sets is equal

stat, p, dof, expected = chi2_contingency(kfold_rmse)
print('freedom degee: dof=%d' % dof)
print('expected:')
print(expected)

# [[10.43478261 18.91304348 30.65217391]
#  [ 5.56521739 10.08695652 16.34782609]]

# Using stat(chisquare) inference
prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
    print('Rejecting the original hypothesis, the network has different prediction accuracy for different test sets')
else:
    print('Cannot rejecti the original hypothesis, the prediction accuracy of the network is independent of the test set')

# Using p-value inference
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Rejecting the original hypothesis, the network has different prediction accuracy for different test sets')
else:
    print('Cannot rejecti the original hypothesis, the prediction accuracy of the network is independent of the test set')

freedom degee: dof=48
expected:
[[16.22766696 16.07089684 15.87297456 15.97291551 15.99055215 15.96899626
  16.11596825 16.00230991 16.71365433 16.27861724 15.85533792 16.19435331
  16.80575677]
 [16.4339214  16.27515872 16.07472085 16.17593205 16.19379285 16.17196298
  16.32080299 16.20570005 16.92608569 16.48551927 16.05686005 16.40018433
  17.01935876]
 [16.92058918 16.75712497 16.55075141 16.65495984 16.67334956 16.65087324
  16.80412093 16.68560938 17.42732824 16.97371505 16.53236168 16.88585304
  17.52336346]
 [16.65485312 16.4939561  16.29082361 16.39339546 16.41149638 16.38937304
  16.54021399 16.42356365 17.1536339  16.70714466 16.27272269 16.62066251
  17.2481609 ]
 [16.57296934 16.41286337 16.21072958 16.31279713 16.33080906 16.30879449
  16.45889383 16.342817   17.06929785 16.62500378 16.19271766 16.53894682
  17.16336011]]
probability=0.950, critical=65.171, stat=0.124
Cannot rejecti the original hypothesis, the prediction accuracy of the network is independent of the test

In [7]:
from scipy.stats import chi2_contingency
kt=chi2_contingency(kfold_rmse)

print('chi2=%.4f, p value=%.4f, freedom degree=%i expected_frep=%s'%kt)


chi2=0.1236, p value=1.0000, freedom degree=48 expected_frep=[[16.22766696 16.07089684 15.87297456 15.97291551 15.99055215 15.96899626
  16.11596825 16.00230991 16.71365433 16.27861724 15.85533792 16.19435331
  16.80575677]
 [16.4339214  16.27515872 16.07472085 16.17593205 16.19379285 16.17196298
  16.32080299 16.20570005 16.92608569 16.48551927 16.05686005 16.40018433
  17.01935876]
 [16.92058918 16.75712497 16.55075141 16.65495984 16.67334956 16.65087324
  16.80412093 16.68560938 17.42732824 16.97371505 16.53236168 16.88585304
  17.52336346]
 [16.65485312 16.4939561  16.29082361 16.39339546 16.41149638 16.38937304
  16.54021399 16.42356365 17.1536339  16.70714466 16.27272269 16.62066251
  17.2481609 ]
 [16.57296934 16.41286337 16.21072958 16.31279713 16.33080906 16.30879449
  16.45889383 16.342817   17.06929785 16.62500378 16.19271766 16.53894682
  17.16336011]]


In [8]:
kt

(0.12355875716276607,
 1.0,
 48,
 array([[16.22766696, 16.07089684, 15.87297456, 15.97291551, 15.99055215,
         15.96899626, 16.11596825, 16.00230991, 16.71365433, 16.27861724,
         15.85533792, 16.19435331, 16.80575677],
        [16.4339214 , 16.27515872, 16.07472085, 16.17593205, 16.19379285,
         16.17196298, 16.32080299, 16.20570005, 16.92608569, 16.48551927,
         16.05686005, 16.40018433, 17.01935876],
        [16.92058918, 16.75712497, 16.55075141, 16.65495984, 16.67334956,
         16.65087324, 16.80412093, 16.68560938, 17.42732824, 16.97371505,
         16.53236168, 16.88585304, 17.52336346],
        [16.65485312, 16.4939561 , 16.29082361, 16.39339546, 16.41149638,
         16.38937304, 16.54021399, 16.42356365, 17.1536339 , 16.70714466,
         16.27272269, 16.62066251, 17.2481609 ],
        [16.57296934, 16.41286337, 16.21072958, 16.31279713, 16.33080906,
         16.30879449, 16.45889383, 16.342817  , 17.06929785, 16.62500378,
         16.19271766, 16.538946